In [1]:
import pickle 

def atisfold(fold):
    assert fold in range(5)
    f = PREFIX + 'atis.fold'+str(fold)+'.pkl'
    train_set, valid_set, test_set, dicts = pickle.load(open(f, 'rb'), encoding='bytes')
    return train_set, valid_set, test_set, dicts

In [2]:
PREFIX = 'dataset/'
w2ne, w2la = {}, {}
train, _, test, dic = atisfold(1)
w2idx, ne2idx, labels2idx = dic[b'words2idx'], dic[b'tables2idx'], dic[b'labels2idx']

In [3]:
print(dic.keys())

dict_keys([b'labels2idx', b'tables2idx', b'words2idx'])


In [4]:

from mitie import *
idx2w  = dict((v,k) for k,v in w2idx.items())
idx2ne = dict((v,k) for k,v in ne2idx.items())
idx2la = dict((v,k) for k,v in labels2idx.items())

test_x,  test_ne,  test_label  = test
train_x, train_ne, train_label = train
trainer = ner_trainer("../MITIE-models/english/total_word_feature_extractor.dat")

output = 0
for sentence_a, label_a in zip(train_x, train_label):
    instance = [idx2w[word].decode('utf8') for word in sentence_a]
    labels = [idx2la[label].decode('utf8') for label in label_a]
    sample = ner_training_instance(instance)
    print(instance)
    print(labels)
    print()
    idx = 0
    last_begin = -1
    entity = ""
    while idx < len(labels):
        if labels[idx].startswith('B'):
            last_begin = idx;
            entity = labels[idx][2:]
        elif labels[idx].startswith('O'):
            if last_begin > 0:
                sample.add_entity(xrange(last_begin, idx), entity)
                last_begin = -1
        idx += 1
    if last_begin > 0:
        sample.add_entity(xrange(last_begin, idx), entity)
    trainer.add(sample)
    output += 1
    if output > 2: break

['what', 'aircraft', 'is', 'used', 'on', 'delta', 'flight', 'DIGITDIGITDIGITDIGIT', 'from', 'kansas', 'city', 'to', 'salt', 'lake', 'city']
['O', 'O', 'O', 'O', 'O', 'B-airline_name', 'O', 'B-flight_number', 'O', 'B-fromloc.city_name', 'I-fromloc.city_name', 'O', 'B-toloc.city_name', 'I-toloc.city_name', 'I-toloc.city_name']

['i', 'want', 'to', 'go', 'from', 'boston', 'to', 'atlanta', 'on', 'monday']
['O', 'O', 'O', 'O', 'O', 'B-fromloc.city_name', 'O', 'B-toloc.city_name', 'O', 'B-depart_date.day_name']

['i', 'need', 'a', 'flight', 'from', 'atlanta', 'to', 'philadelphia', 'and', 'i', "'m", 'looking', 'for', 'the', 'cheapest', 'fare']
['O', 'O', 'O', 'O', 'O', 'B-fromloc.city_name', 'O', 'B-toloc.city_name', 'O', 'O', 'O', 'O', 'O', 'O', 'B-cost_relative', 'O']



In [5]:
trainer.num_threads = 4

ner = trainer.train()


In [6]:
print ("tags:", ner.get_possible_ner_tags())

tags: ['airline_name', 'flight_number', 'fromloc.city_name', 'toloc.city_name', 'depart_date.day_name', 'cost_relative']


In [7]:
tokens = ['i', 'need', 'a', 'flight', 'from', 'toronto', 'to', 'montreal', 'reaching', 'montreal', 'early', 'on', 'friday']
entities = ner.extract_entities(tokens)

print ("\nEntities found:", entities)
print ("\nNumber of entities detected:", len(entities))
for e in entities:
    range = e[0]
    tag = e[1]
    entity_text = " ".join(tokens[i] for i in range)
    print ("    " + tag + ": " + entity_text)


Entities found: [(range(5, 6), 'fromloc.city_name', 0.6331889743239222), (range(7, 8), 'toloc.city_name', 0.42296601666364264), (range(9, 10), 'toloc.city_name', 0.12189891205777652), (range(12, 13), 'depart_date.day_name', 0.3760252247693764)]

Number of entities detected: 4
    fromloc.city_name: toronto
    toloc.city_name: montreal
    toloc.city_name: montreal
    depart_date.day_name: friday


Here is the MITIE tool output information.


    Training to recognize 6 labels: 'airline_name', 'flight_number', 'fromloc.city_name', 'toloc.city_name', 'depart_date.day_name', 'cost_relative'
    Part I: train segmenter
    words in dictionary: 200000
    num features: 271
    now do training
    C:           20
    epsilon:     0.01
    num threads: 4
    cache size:  5
    max iterations: 2000
    loss per missed segment:  3
    C: 20   loss: 3 	0.428571
    C: 35   loss: 3 	0.428571
    C: 20   loss: 4.5 	0.428571
    C: 5   loss: 3 	0.428571
    C: 20   loss: 1.5 	0.428571
    C: 21   loss: 3 	0.428571
    C: 20   loss: 3.1 	0.428571
    C: 19   loss: 3 	0.428571
    C: 20   loss: 3 	0.428571
    best C: 20
    best loss: 3
    num feats in chunker model: 4095
    train: precision, recall, f1-score: 1 1 1 
    Part I: elapsed time: 1 seconds.

    Part II: train segment classifier
    now do training
    num training samples: 10
    test on train: 
    1 0 0 0 0 0 
    0 1 0 0 0 0 
    0 0 3 0 0 0 
    0 0 0 3 0 0 
    0 0 0 0 1 0 
    0 0 0 0 0 1 

    overall accuracy: 1
    Part II: elapsed time: 9 seconds.
    df.number_of_classes(): 6

